In [ ]:
# !pip install cloudpickle

In [ ]:
# from gevent import monkey; monkey.patch_all()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import syft as sy

In [ ]:
from syft import Worker

In [ ]:
worker = Worker()
client = sy.new_login(node=worker, email="info@openmined.org", password="changethis")

In [ ]:
client.api

In [ ]:
%%time
client.api.services.executor.run_task(1.2)

In [ ]:
print("test")

In [ ]:
client.api.services.executor.get_results()

In [ ]:
from gevent import monkey; monkey.patch_all()

In [ ]:
import gevent
import gipc
import time

In [ ]:
monkey.patch_all()

In [ ]:
def cpu_bound_func(total: int = 250_000_00) -> None:
    import numpy as np
    x = np.array([range(total)])
    y = x.sum()
    print("Did work", y)
    return y

In [ ]:
%%time
cpu_bound_func(250_000_00)

In [ ]:
def runner(reader):
    print("type of reader", type(reader)) 
    with reader:
        try:
            iters = reader.get()
            print("got iters", iters)
            result = cpu_bound_func(iters)
            print("got result", result)
            reader.put(result)
            reader.close()
        except Exception as e:
            print("got excpection in process", e)
            reader.close()

In [ ]:
def writer(writer, data):
    print("writer", "data", writer, data)
    with writer:
        writer.put(data)
        gevent.sleep(1)
        result = writer.get()
        writer.close()
        print("got result in greenthread", result)
    return result

In [ ]:
def task(pid, arg, db):
    print("got arg in taks", arg)
    """
    Some non-deterministic task
    """
    # cpu_bound_func()
    # with gipc.pipe() as (r, w):
    with gipc.pipe(duplex=True) as (cend, pend):

        p = gipc.start_process(runner, args=(cend,))
        wg = gevent.spawn(writer, pend, arg)
        print("what is wg", wg)
        # wg.join()
        print("wg value", wg.value)
        try:
            p.join()
        except KeyboardInterrupt:
            wg.kill(block=True)
            p.terminate()
        p.join()
    
    print("end", wg.value)
    db[pid] = wg.value
    return wg.value

In [ ]:
db = {}
process_count = 10
def asynchronous():
    data = 500_000_00
    threads = [gevent.spawn(task, i, data, db) for i in range(process_count)]
    # gevent.joinall(threads)
    # results = [thread.value for thread in threads]
    # print("final results", results)

print('Asynchronous:')
asynchronous()

In [ ]:
while len(db.keys()) < process_count:
    print(".", end="")
    time.sleep(1)
    print("len", len(db.keys()))
print("results arrived", db)

In [ ]:
db